<a href="https://colab.research.google.com/github/soan12345/OCR/blob/main/BEST_OCR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pdfplumber

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.5/42.5 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 32.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 14.6 MB/s eta 0:00:00


In [ ]:
!apt install tesseract-ocr

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 49 not upgraded.
Need to get 4,816 kB of archives.
After this operation, 15.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-eng all 1:4.00~git30-7274cfa-1.1 [1,591 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-osd all 1:4.00~git30-7274cfa-1.1 [2,990 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr amd64 4.1.1-2.1build1 [236 kB]
Fetched 4,816 kB in 2s (2,933 kB/s)
Selecting previously unselected package tesseract-ocr-eng.
(Reading database ... 124565 files and directories currently installed.)
Preparing to unpack .../tesseract-ocr-

In [ ]:
!pip install pytesseract pdf2image pillow opencv-python-headless numpy pandas
!pip install PyPDF2
!apt-get install -y poppler-utils tesseract-ocr


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 49 not upgraded.
Need to get 186 kB of archives.
After this operation, 696 kB of additional disk space will be used.
Ign:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.5
Err:1 http://security.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.5
  404  Not Found [IP: 185.125.190.83 80]
E: Failed to fetch http://security.ubuntu.com/ubuntu/pool/main/p/poppler/poppler-utils_22.02.0-2ubuntu0.5_amd64.deb  404  Not Found [IP: 185.125.190.83 80]
E: Unable to fetch some archives, maybe run apt-get update or try with --fix-missing?


In [ ]:
#code to extract Delhi bill format

import pytesseract
from pdf2image import convert_from_path
import json
import re
from PIL import Image
import os
import cv2
import numpy as np

# Set Tesseract OCR path (modify this if needed for your environment)
pytesseract.pytesseract.tesseract_cmd = r'/usr/bin/tesseract'  # Update for your OS

# Function to preprocess and perform OCR on specific regions
def extract_text_from_pdf(pdf_path):
    """Extracts text from a PDF using OCR."""
    try:
        pages = convert_from_path(pdf_path, dpi=300)  # Adjust DPI for better OCR accuracy
        extracted_text = ""

        for page_num, page in enumerate(pages):
            # Convert PIL Image to OpenCV Image
            open_cv_image = cv2.cvtColor(np.array(page), cv2.COLOR_RGB2BGR)

            # Preprocessing: Convert to grayscale and apply adaptive threshold
            gray = cv2.cvtColor(open_cv_image, cv2.COLOR_BGR2GRAY)
            processed_image = cv2.adaptiveThreshold(
                gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 2
            )

            # Perform OCR on the preprocessed image
            text = pytesseract.image_to_string(processed_image, config="--psm 6")
            extracted_text += f"Page {page_num + 1}:\n{text}\n"

        return extracted_text
    except Exception as e:
        print(f"Error during PDF processing: {e}")
        return ""

# Function to parse the extracted text and structure it into JSON
def parse_extracted_text(text):
    # Define all the regex patterns
    patterns = {
        "name": r"Name\s*:\s*([\s\S]+?)(?=\nBilling Address)",
        "billing_address": r"Billing Address\s*:\s*([\s\S]+?\w+-)()",
        "mobile_no": r"Mobile\s*/\s*Tel\.?\s*No\.\s*:\s*(\d{10})",
        "email_id": r"Email ID\s*:\s*([a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,})",
        "district": r"District\s*/\s*Division\s*:\s*([A-Za-z\s]+)",
        "walking_sequence": r"Walking Sequence\s*:\s*([A-Z0-9]+)",
        "bill_month": r"Bill Month\s*:\s*([A-Z]{3}-\d{2})",
        "bill_date": r"Bill Date\s*:\s*(\d{2}-\d{2}-\d{4})",
        "sanctioned_load": r"Sanctioned Load\s*:\s*([\d\.]+)\s*\(kVA\)",
        "contract_demand": r"Contract Demand\s*:\s*([\d\.]+)",
        "power_factor": r"Power Factor\s*:\s*([\d\.]+)",
        "pole_no": r"Pole No\.?\s*:\s*([\w\d]+)",
        "meter_reading_status": r"Meter Reading Status\s*:\s*([\w]+)",
        "cycle_no": r"Cycle No\.?\s*\s*(\d+)",
        "ca_no": r"CA No\.\s*(\d+)",
        "energisation_date": r"Energisation Date\s*\:\s*([\d]{2}\.[\d]{2}\.[\d]{4})",
        "meter_type": r"Meter Type\s*\:\s*([A-Z0-9]+)",
        "supply_type": r"Supply Type\s*\:\s*([A-Z]+)",
        "bill_no": r"Bill No\.\s*(\d+)",
        "bill_basis": r"Bill Basis\s*\:\s*([A-Za-z]+)",
        "od_no": r"O\.D\. No\.\s*\:\s*([A-Za-z/0-9]+)",
        "cctv_tagged_no": r"CCTV Tagged No\s*([A-Za-z]*)",  # Assuming it can be empty or have a value
        "street_light_tagged_no": r"Street Light Tagged No\s*([A-Za-z]*)",  # Assuming it can be empty or have a value
        "wifi_tagged": r"WI-FI Tagged\s*([A-Z])",
        #"bill_amount_payable": r"Bill Amount Payable\s*
        "due_date_of_payment": r"Due Date of Payment\s*\n\s*([\d]{2}-[\d]{2}-[\d]{4})"
    }

    parsed_data = {
        "customer_details": {},
        "bill_details": {},
    }

    # Apply regex patterns to the extracted text
    for field, pattern in patterns.items():
        match = re.search(pattern, text, re.IGNORECASE)
        value = match.group(1).strip() if match else None
        if field in ["name", "billing_address", "mobile_no", "email_id", "district"]:
            parsed_data["customer_details"][field] = value
        else:
            parsed_data["bill_details"][field] = value

    return parsed_data

# Main function
def main(pdf_path, output_path="output.json"):
    """Extracts text from a PDF, parses it, and saves the structured data to a JSON file."""
    if not os.path.exists(pdf_path):
        print(f"PDF file not found: {pdf_path}")
        return

    # Step 1: Extract text from PDF
    print("Extracting text from PDF...")
    extracted_text = extract_text_from_pdf(pdf_path)

    if not extracted_text:
        print("No text extracted. Exiting.")
        return

    # Step 2: Parse extracted text
    print("Parsing extracted text...")
    parsed_data = parse_extracted_text(extracted_text)

    # Step 3: Save the structured data to a JSON file
    try:
        with open(output_path, "w") as json_file:
            json.dump(parsed_data, json_file, indent=4)
        print(f"Parsed data saved to {output_path}")
    except Exception as e:
        print(f"Error saving JSON: {e}")

    # Optional: Print parsed data to console for verification
    print("Extracted and parsed data:")
    print(json.dumps(parsed_data, indent=4))

# Replace with the path to your PDF file
pdf_path = "/content/11304_DelhiNCR_103424503_10-Aug-24 (1).pdf"  # Update with the actual PDF path
main(pdf_path, "output.json")

Extracting text from PDF...
Parsing extracted text...
Parsed data saved to output.json
Extracted and parsed data:
{
    "customer_details": {
        "name": "DRGUPTA & ANIL KUMAR",
        "billing_address": "LT Mr. JATTU RAM&D R GUPTA SHOP Sanctioned Load :5.00 (kVA) CA No. 7103424503\n. Energisation Date :05.10.2010\nAT G-9 GF PLOT NO 5 SEC 12 KRISHNA MALL Contract Demand : Meter Type -1PSK\nDWARKA NEAR KRISHNA MALL NEW DELHI 110078 MDI : 00 Supply Type \u2018LT\nPower Factor 1.000 Bill No. 100788028110\nPole No. :DWKNNPO0SO op aa on 0432782830\n. .D. No. :\nMeter Reading Status :DL CCTV Tagged \u201cNo\nMobile/Tel. No. :9971847863 Cycle No. 13 Street Light Tagged :No\nEmail ID : drgupta81@gmail.com WEFT Tagged \u2018No\nDistrict / Division \u2014 :Dwarka Tariff Category | :Non.Domestic [ LT ] (Up to 10 kw)\nWalking Sequence :S12M15393A0AA\nBill Month :AUG-24\nBill Date :10-08-2024 Customer Care Centre No, 19123 (24x7 Toll Free\nMeter No Units Billed Consumption (Current) Billed Con

In [ ]:
#Cropping the img and then performing OCR
import pdf2image
import cv2
import numpy as np
import pytesseract
from PIL import Image
import re
import json
from typing import List, Dict, Any, Tuple
import os


pytesseract.pytesseract.tesseract_cmd = r'/usr/bin/tesseract'

class PDFTableExtractor:


    def convert_pdf_to_images(self, pdf_path: str, dpi: int = 300) -> List[Image.Image]:
        """
        Convert PDF to list of PIL Images
        """
        try:
            return pdf2image.convert_from_path(pdf_path, dpi=dpi)
        except Exception as e:
            print(f"Error converting PDF to images: {str(e)}")
            return []

    def preprocess_image(self, image: np.ndarray) -> np.ndarray:
        """
        Preprocess image for better table detection
        """
        # Convert to grayscale
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

        # Binary threshold
        _, binary = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

        # Dilate to connect text
        kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
        dilated = cv2.dilate(binary, kernel, iterations=3)

        return dilated

    def detect_tables(self, image: np.ndarray) -> List[Tuple[int, int, int, int]]:
        """
        Detect tables in the image and return their coordinates
        """
        # Find contours
        contours, _ = cv2.findContours(image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        # Filter contours based on area
        min_area = image.shape[0] * image.shape[1] * 0.01  # 1% of image area
        table_contours = [cnt for cnt in contours if cv2.contourArea(cnt) > min_area]

        # Get bounding rectangles
        tables = []
        for cnt in table_contours:
            x, y, w, h = cv2.boundingRect(cnt)
            tables.append((x, y, x+w, y+h))

        return tables

    def crop_tables(self, image: Image.Image, coordinates: List[Tuple[int, int, int, int]]) -> List[Image.Image]:
        """
        Crop tables from the image using coordinates
        """
        return [image.crop(coord) for coord in coordinates]

    def perform_ocr(self, image: Image.Image) -> str:
        """
        Perform OCR on an image
        """
        try:
            # Configure OCR for better table recognition
            custom_config = r'--oem 3 --psm 6'
            return pytesseract.image_to_string(image, config=custom_config)
        except Exception as e:
            print(f"OCR Error: {str(e)}")
            return ""

    def parse_table_text(self, text: str) -> Dict[str, Any]:
        """
        Parse extracted text into structured data
        """
        # Define patterns for different sections
        patterns = {
            'bill_details': {
                'bill_for': r'Bill For\s*:\s*([A-Za-z]+-\d{4})',
                'date_of_bill': r'Date of Bill\s*:\s*(\d{2}/\d{2}/\d{4})',
                'invoice_no': r'Invoice No\.\s*:\s*(\d+)',
                'bill_period': r'Bill Period\s*:\s*(\d{2}/\d{2}/\d{4})\s*-\s*(?:\n|\r\n)*\s*(\d{2}/\d{2}/\d{4})',
                  "book_folio_no": r"Book Folio No\s*\.?\s*:\s*(\d+)",
        "consumer_no": r"Consumer No\s*\.?\s*:\s*([\d\-X]+)",
        "c_a_no": r"C\.?A\.? No\s*\.?\s*:\s*(\d+)",
        "cycle": r"Cycle\s*:\s*([\w]+)",
        "type_of_supply": r"Type of Supply\s*:\s*([\w]+)",
        "service_no": r"Service No\s*\.?\s*:\s*([\w\-]+)",
        "installation_no": r"Installation No\s*\.?\s*:\s*(\d+)",
        "sanctioned_load": r"Sanctioned Load\s*:\s*([\d\.]+)\s*KW",
        "security_deposit": r"Security Deposit\s*:\s*([\d\.]+)",
        "last_payment_received": r"Last Payment Received\s*₹?\s*([\d\.]+)",
        "last_payment_received_date": r"Last Payment Received Date\s*:\s*([\d/]+)",
        "bill_period_from": r"Bill Period From\s*:\s*([\d/]+)",
        "bill_period_to": r"Bill Period To\s*:\s*([\d/]+)",
        "tariff": r"Tariff\s*:\s*(\S+)",
        "category": r"Category\s*:\s*(\S+)",
        "ward": r"Ward\s*:\s*(\S+)",
            },
            'customer_details': {
                "name": r"Name\s*:\s*([\s\S]+?)(?=\nMobile No|Mobile No)",
        "billing_address": r"Billing Address\s*:\s*([\s\S]+?\w+-\d{6})",
        "power_supply_address": r"Power Supply Address\s*:\s*([\s\S]+?\w+-\d{6})",
        "mobile_no": r"Mobile No\s*:\s*(\d+[X]*\d+)",
        "email_id": r"Email ID\s*:\s*([a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,})",
            }
        }

        parsed_data = {}
        for section, section_patterns in patterns.items():
            parsed_data[section] = {}
            for field, pattern in section_patterns.items():
                match = re.search(pattern, text)
                if match:
                    if field == 'bill_period':
                        parsed_data[section]['bill_period_from'] = match.group(1)
                        parsed_data[section]['bill_period_to'] = match.group(2)
                    else:
                        parsed_data[section][field] = match.group(1)

        return parsed_data

    def process_pdf(self, pdf_path: str, output_path: str = None) -> Dict[str, Any]:
        """
        Process PDF and extract table data
        """
        # Convert PDF to images
        images = self.convert_pdf_to_images(pdf_path)
        if not images:
            return {}

        all_data = []
        for idx, pil_image in enumerate(images):
            # Convert PIL Image to OpenCV format
            opencv_image = cv2.cvtColor(np.array(pil_image), cv2.COLOR_RGB2BGR)

            # Preprocess image
            processed = self.preprocess_image(opencv_image)

            # Detect tables
            table_coordinates = self.detect_tables(processed)

            # Crop tables
            table_images = self.crop_tables(pil_image, table_coordinates)

            # Process each table
            for table_idx, table_image in enumerate(table_images):
                # Perform OCR
                table_text = self.perform_ocr(table_image)

                # Parse text
                parsed_data = self.parse_table_text(table_text)
                all_data.append(parsed_data)

                # Save cropped table image for debugging
                if output_path:
                    table_image.save(f"{output_path}/table_{idx}_{table_idx}.png")

        # Combine all extracted data
        combined_data = self.combine_data(all_data)

        # Save to JSON if output path provided
        if output_path:
            with open(f"{output_path}/extracted_data.json", 'w', encoding='utf-8') as f:
                json.dump(combined_data, f, indent=2, ensure_ascii=False)

        return combined_data

    def combine_data(self, data_list: List[Dict[str, Any]]) -> Dict[str, Any]:
        """
        Combine data from multiple tables into a single structure
        """
        combined = {
            "bill_details": {},
            "customer_details": {},
        }

        for data in data_list:
            for section in data:
                combined[section].update(data[section])

        return combined

# Example usage
if __name__ == "__main__":
    # Initialize extractor
    extractor = PDFTableExtractor()

    # Process PDF
    pdf_path = "/content/'bill_202410_604571021_en' (1) (1) (1) (1).pdf"
    output_dir = "output"

    # Create output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)

    # Extract data
    result = extractor.process_pdf(pdf_path, output_dir)

    # Print results
    print(json.dumps(result, indent=2))

{
  "bill_details": {
    "bill_for": "Oct-2024",
    "date_of_bill": "16/10/2024",
    "invoice_no": "410604571021",
    "consumer_no": "604-571-021",
    "sanctioned_load": "418.980",
    "tariff": "LTIB",
    "category": "RESIDENTIAL"
  },
  "customer_details": {
    "name": "Bill For : Oct-2024 Date of Bill : 16/10/2024 Invoice No. : 410604571021\n\nMRS P J & SWATIL & J P GADA ;\n",
    "billing_address": "Type of Supply ; 3p Bill Period : 06/09/2024 -\n\n0-4-B, FLOOR-2,PLOT-3A,GURDEVI MANSION, K A Service No ; ye\n\nSUBRAMANYAM MARG, BRAHMANWADA KINGS . 312229-X-X . 08/10/2024\n\nCIRCLE,MATUNGA,MUMBAI-400019",
    "power_supply_address": "5 ity D it Ward\n\n0-4-B, FLOOR-2,PLOT-3A,GURDEVI MANSION, K A ecurity VEposit : 3856.00 ar : FN\n\nSUBRAMANYAM MARG, BRAHMANWADA KINGS 5 \u00ab\n\nCIRCLE,MATUNGA, MUMBAI-400019",
    "mobile_no": "98XXXXX156",
    "email_id": "XXXXXXXXada@gmail.com"
  }
}


In [ ]:
!apt-get install -y poppler-utils


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libpoppler-dev libpoppler-private-dev libpoppler118
Recommended packages:
  poppler-data
The following NEW packages will be installed:
  poppler-utils
The following packages will be upgraded:
  libpoppler-dev libpoppler-private-dev libpoppler118
3 upgraded, 1 newly installed, 0 to remove and 54 not upgraded.
Need to get 1,462 kB of archives.
After this operation, 696 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 libpoppler-private-dev amd64 22.02.0-2ubuntu0.6 [199 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 libpoppler-dev amd64 22.02.0-2ubuntu0.6 [5,184 B]
Get:3 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 libpoppler118 amd64 22.02.0-2ubuntu0.6 [1,071 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.

In [ ]:
import camelot
import json
import os

def get_all_table_coordinates(file_path, page_number):
    # Read the PDF and extract tables from the specified page
    tables = camelot.read_pdf(file_path, pages=str(page_number))

    # Initialize a list to store the coordinates of all tables
    table_coordinates = []

    # Iterate through each table detected
    for i, table in enumerate(tables):
        # Extract the coordinates from the first and last rows of the table
        top_left = table.df.iloc[0, 0]
        bottom_right = table.df.iloc[-1, 0]
        # Append the coordinates to the list in a dictionary format
        table_coordinates.append({
            "table_name": f"Table {i+1}",
            "top_left": top_left,
            "bottom_right": bottom_right
        })

    return table_coordinates

# Define file path and page number
file_path = "/content/604571021_2408 (1).pdf"
page_number = 1

# Get table coordinates
coordinates = get_all_table_coordinates(file_path, page_number)

# Automatically generate output file path based on input PDF file name
base_name = os.path.splitext(os.path.basename(file_path))[0]
output_file_path = f"/content/{base_name}_table_coordinates.json"

# Save the coordinates to a JSON file
with open(output_file_path, 'w') as json_file:
    json.dump(coordinates, json_file, indent=4)
print(coordinates)
print(f"Table coordinates saved to {output_file_path}")


[{'table_name': 'Table 1', 'top_left': 'Name :\nMRS P J & SWATI L & J P GADA\nMobile No:98XXXXX156\nEmail ID:XXXXXXXXada@gmail.com\nBilling Address :\n0-4-B,FLOOR-2,PLOT-3A,GURDEVI MANSION,K A\nSUBRAMANYAM MARG,BRAHMANWADA KINGS\nCIRCLE,MATUNGA,MUMBAI-400019\nPower Supply Address :\n0-4-B,FLOOR-2,PLOT-3A,GURDEVI MANSION,K A\nSUBRAMANYAM MARG,BRAHMANWADA KINGS\nCIRCLE,MATUNGA,MUMBAI-400019', 'bottom_right': ''}, {'table_name': 'Table 2', 'top_left': 'Past Consumption\nBar Graph\nUnit\nMonth\nKWH\nMeter No -\nN208524\n277\nJul-24\n274\nJun-24\n407\nMay-24\n244\nApr-24\n59\nMar-24\n0\nFeb-24\n285\nJan-24\n185\nDec-23\n343\nNov-23\n226\nOct-23\n207\nSep-23', 'bottom_right': 'Aug-24\n138\nAug-23\n133'}]
Table coordinates saved to /content/604571021_2408 (1)_table_coordinates.json


In [ ]:
!pip install PyMuPDF(library )

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 49.3 MB/s eta 0:00:00


In [ ]:
import fitz  # PyMuPDF

# Open the PDF
pdf_document = "/content/604571021_2408 (1).pdf"
pdf = fitz.open(pdf_document)

# Select the page you want
page = pdf[0]  # First page (0-indexed)

# Extract all text with bounding box information
blocks = page.get_text("blocks")  # Returns a list of text blocks with coordinates

# Display blocks and their coordinates
for block in blocks:
    x1, y1, x2, y2, text = block[0], block[1], block[2], block[3], block[4]
    print(f"Block Text: {text.strip()}")
    print(f"Coordinates: x1={x1}, y1={y1}, x2={x2}, y2={y2}")
    print("-" * 50)


Block Text: Customer Care `F/N` Ward,B.E.S & T
Coordinates: x1=456.92999267578125, y1=49.168128967285156, x2=541.0651245117188, y2=54.168128967285156
--------------------------------------------------
Block Text: Undertaking,New Ancilliary Bldg,6th
Floor,Opp.Ambedkar College,Wadala,Mumbai-
Coordinates: x1=447.1499938964844, y1=54.978126525878906, x2=550.8449096679688, y2=65.77811431884766
--------------------------------------------------
Block Text: 400031.TelNo-24148662,Ext 676
Coordinates: x1=463.1600036621094, y1=66.5881118774414, x2=534.8298950195312, y2=71.5881118774414
--------------------------------------------------
Block Text: Name :
Coordinates: x1=45.0, y1=89.61096954345703, x2=70.64799499511719, y2=97.61096954345703
--------------------------------------------------
Block Text: This Electric Bill is issued for electricity used and may not be treated as proof for other
Coordinates: x1=24.756206512451172, y1=175.7899169921875, x2=34.75620651245117, y2=569.0
----------------

In [ ]:
import camelot


tables = camelot.read_pdf('/content/604571021_2408 (1).pdf', pages='1,2', flavor='stream')

# Checking the extracted tables
for i, table in enumerate(tables):
    if i == 0:
        print(f"Personal Info Table (Table {i+1}):")
    elif i == 1:
        print(f"Bill Details Table (Table {i+1}):")
    elif i == 2:
        print(f"Additional Details Table (Table {i+1}):")
    else:
        print(f"Table {i+1}:")
    print(table.df)

# Extract the first table (personal info)
personal_info_table = tables[0].df
# Extract the second table (bill details)
bill_details_table = tables[1].df
# Extract the third table (additional details)
additional_details_table = tables[2].df


# Selecting specific columns from each table
personal_info_columns = personal_info_table[[0, 1]]  # Assuming name, mobile, email are in columns 0 and 1
bill_details_columns = bill_details_table[[0, 1, 2]]  # invoice number, sanction rules
additional_columns = additional_details_table[[0, 1]]


# Printing selected columns
print("\nSelected Personal Info Columns:")
print(personal_info_columns)

print("\nSelected Bill Details Columns:")
print(bill_details_columns)

print("\nSelected Additional Details Columns:")
print(additional_columns)

import pandas as pd
combined_df = pd.concat([personal_info_columns, bill_details_columns, additional_columns], axis=1)

print("\nCombined DataFrame:")
print(combined_df)


Personal Info Table (Table 1):
                                            0  \
0                                               
1                                               
2                                               
3                                               
4                                      Name :   
5                MRS P J & SWATI L & J P GADA   
6                                               
7                                               
8                        Mobile No:98XXXXX156   
9              Email ID:XXXXXXXXada@gmail.com   
10                                              
11                          Billing Address :   
12                                              
13  0-4-B,FLOOR-2,PLOT-3A,GURDEVI MANSION,K A   
14                                              
15         SUBRAMANYAM MARG,BRAHMANWADA KINGS   
16                                              
17               CIRCLE,MATUNGA,MUMBAI-400019   
18                                    

In [ ]:
import camelot
import json
import os

def get_all_table_coordinates(file_path, page_number):
    # Read the PDF and extract tables from the specified page
    tables = camelot.read_pdf(file_path, pages=str(page_number))

    # Initialize a list to store the coordinates of all tables
    table_coordinates = []

    # Iterate through each table detected
    for i, table in enumerate(tables):
        # Extract the coordinates from the first and last rows of the table
        top_left = table.df.iloc[0, 0]
        bottom_right = table.df.iloc[-1, 0]
        # Append the coordinates to the list in a dictionary format
        table_coordinates.append({
            "table_name": f"Table {i+1}",
            "top_left": top_left,
            "bottom_right": bottom_right
        })

    return table_coordinates

# Define file path and page number
file_path = "/content/604571021_2408 (1).pdf"
page_number = 1

# Get table coordinates
coordinates = get_all_table_coordinates(file_path, page_number)

# Automatically generate output file path based on input PDF file name
base_name = os.path.splitext(os.path.basename(file_path))[0]
output_file_path = f"/content/{base_name}_table_coordinates.json"

# Save the coordinates to a JSON file
with open(output_file_path, 'w') as json_file:
    json.dump(coordinates, json_file, indent=4)

print(f"Table coordinates saved to {output_file_path}")


Table coordinates saved to /content/604571021_2408 (1)_table_coordinates.json


In [ ]:
import camelot
import json
import os
import re

def extract_fields(text):
    # Define patterns for each field
    patterns = {
            "name": r"Name\s*:\s*([\s\S]+?)(?=\nMobile No|Mobile No)",
        "billing_address": r"Billing Address\s*:\s*([\s\S]+?\w+-\d{6})",
        "power_supply_address":  r"Power Supply Address\s*:\s*([\s\S]+?\w+-\d{6})",
        "mobile_no": r"Mobile No\s*:\s*(\d+[X]*\d+)",
        "email_id": r"Email ID\s*:\s*([a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,})",
        "bill_for": r"Bill For\s*:\s*([A-Za-z]+\s*-\s*\d{4})",
        "date_of_bill": r"Date of Bill\s*:\s*([\d/]+)",
        "invoice_no": r"Invoice No\s*\.?\s*:\s*(\d+)",
        "book_folio_no": r"Book Folio No\s*\.?\s*:\s*(\d+)",
        "consumer_no": r"Consumer No\s*\.?\s*:\s*([\d\-X]+)",
        "c_a_no": r"C\.?A\.? No\s*\.?\s*:\s*(\d+)",
        "cycle": r"Cycle\s*:\s*(\d+)",
        "type_of_supply": r"Type of Supply\s*:\s*([\w]+)",
        "service_no": r"Service No\s*\.?\s*:\s*([\w\-]+)",
        'installation_no': r'Installation No:\s*(.*)',
        'sanctioned_load': r'Sanctioned Load:\s*(.*)',
        'security_deposit': r'Security Deposit:\s*(.*)',
        'last_payment': r'Last Payment Received:\s*(.*)',
        'last_payment_date': r'Last Payment Received Date:\s*(.*)',
        'bill_period': r'Bill Period:\s*(.*)',
        'tariff': r'Tariff:\s*(.*)',
        'category': r'Category:\s*(.*)',
        'ward': r'Ward:\s*(.*)',
        'current_bill_amount': r'Current Bill Amount:\s*(.*)',
        'past_dues': r'Past Dues:\s*(.*)',
        'due_date': r'Due Date:\s*(.*)',
        'amount_before_due': r'Bill Amount Before Due Date:\s*(.*)',
        'amount_after_due': r'Bill Amount After Due Date:\s*(.*)'
    }

    # Extract fields using patterns
    extracted_data = {}
    for field, pattern in patterns.items():
        match = re.search(pattern, text)
        if match:
            extracted_data[field] = match.group(1).strip()

    return extracted_data

def get_all_table_coordinates_and_text(file_path, page_number):
    # Read the PDF and extract tables from the specified page
    tables = camelot.read_pdf(file_path, pages=str(page_number))

    # Initialize a list to store the coordinates of all tables
    table_coordinates = []

    # Initialize variable to store extracted text
    extracted_text = ""

    # Iterate through each table detected
    for i, table in enumerate(tables):
        # Extract text from the table
        for index, row in table.df.iterrows():
            extracted_text += " ".join(row) + " "

        # Extract the coordinates from the first and last rows of the table
        top_left = table.df.iloc[0, 0]
        bottom_right = table.df.iloc[-1, 0]

        # Append the coordinates to the list in a dictionary format
        table_coordinates.append({
            "table_name": f"Table {i+1}",
            "top_left": top_left,
            "bottom_right": bottom_right
        })

    # Extract structured data from the text
    structured_data = extract_fields(extracted_text)

    return table_coordinates, structured_data

# Define file path and page number
file_path = "/content/604571021_2408 (1).pdf"
page_number = 1

# Get table coordinates and extracted data
coordinates, structured_data = get_all_table_coordinates_and_text(file_path, page_number)

# Generate output file paths
base_name = os.path.splitext(os.path.basename(file_path))[0]
coordinates_file_path = f"/content/{base_name}_table_coordinates.json"
extracted_data_file_path = f"/content/{base_name}_extracted_data.json"

# Save the coordinates to a JSON file
with open(coordinates_file_path, 'w') as json_file:
    json.dump(coordinates, json_file, indent=4)

# Save the extracted data to a JSON file
with open(extracted_data_file_path, 'w') as json_file:
    json.dump(structured_data, json_file, indent=4)

print("Table coordinates saved to", coordinates_file_path)
print("Extracted data saved to", extracted_data_file_path)
print("\nExtracted Data:")
print(json.dumps(structured_data, indent=4))

Table coordinates saved to /content/604571021_2408 (1)_table_coordinates.json
Extracted data saved to /content/604571021_2408 (1)_extracted_data.json

Extracted Data:
{
    "name": "MRS P J & SWATI L & J P GADA",
    "billing_address": "0-4-B,FLOOR-2,PLOT-3A,GURDEVI MANSION,K A\nSUBRAMANYAM MARG,BRAHMANWADA KINGS\nCIRCLE,MATUNGA,MUMBAI-400019",
    "power_supply_address": "0-4-B,FLOOR-2,PLOT-3A,GURDEVI MANSION,K A\nSUBRAMANYAM MARG,BRAHMANWADA KINGS\nCIRCLE,MATUNGA,MUMBAI-400019",
    "mobile_no": "98XXXXX156",
    "email_id": "XXXXXXXXada@gmail.com"
}


In [ ]:
!pip install camelot-py

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.6/66.6 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 87.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 76.2 MB/s eta 0:00:00


Extract tables and then we can create array and then do key-value pair

In [ ]:
data = {
    "table_number": 2,
    "rows": 2,
    "cols": 5,
    "data": [
        [
            "Current Bill Amount",
            "Past Dues",
            "Due Date",
            "Bill Amount Before Due Date",
            "Bill Amount After Due Date"
        ],
        [
            "1165.05",
            "7.71",
            "09/09/2024",
            "1170.00",
            "1188"
        ]
    ]
}

# Converting table data into key-value pairs
key_value_pairs = dict(zip(data["data"][0], data["data"][1]))

print(key_value_pairs)


{'Current Bill Amount': '1165.05', 'Past Dues': '7.71', 'Due Date': '09/09/2024', 'Bill Amount Before Due Date': '1170.00', 'Bill Amount After Due Date': '1188'}


In [ ]:
import camelot
import json

def extract_tables_from_pdf(file_path, page_number):
    """Extract and format tables from PDF"""
    tables = camelot.read_pdf(
        file_path,
        pages=str(page_number),
        flavor='lattice',
        line_scale=40,
        strip_text='\n'
    )

    formatted_tables = []
    structured_data = {}

    for i, table in enumerate(tables):
        df = table.df  # Convert table to pandas DataFrame
        table_dict = {
            'table_number': i + 1,
            'rows': len(df),
            'cols': len(df.columns),
            'data': df.values.tolist()
        }
        formatted_tables.append(table_dict)

        # Convert table data into key-value pairs dynamically
        if len(df) > 1:  # Ensure table has at least header and one row
            headers = df.iloc[0].tolist()
            values = df.iloc[1].tolist()
            structured_data[f'Table_{i+1}'] = dict(zip(headers, values))

    return tables, formatted_tables, structured_data


def extract_fields_from_pdf(file_path, page_number):
    """Extract structured key-value data from tables"""
    tables, formatted_tables, structured_data = extract_tables_from_pdf(file_path, page_number)
    extracted_data = {'tables': formatted_tables, 'structured_data': structured_data}
    return extracted_data


def display_tables(tables):
    """Display tables in a readable format"""
    print("\nExtracted Tables:")
    print("-" * 50)
    for i, table in enumerate(tables):
        print(f"\nTable {i+1}:")
        print(table.df.to_string())
        print("\nAccuracy Score:", table.accuracy)
        print("-" * 50)


def main():
    file_path = "/content/604571021_2408.pdf"
    page_number = "1,2"

    try:
        # Extract tables and structured data
        tables, _, structured_data = extract_tables_from_pdf(file_path, page_number)

        # Display tables
        display_tables(tables)

        # Save extracted structured data to JSON
        output_file = 'extracted_fields.json'
        with open(output_file, 'w', encoding='utf-8') as f:
            json.dump(structured_data, f, indent=4, ensure_ascii=False)

        print(f"\nStructured data extracted and saved to {output_file}")
        print("\nExtracted Fields:")
        print(json.dumps(structured_data, indent=2))

    except Exception as e:
        print(f"Error processing PDF: {str(e)}")


if __name__ == "__main__":
    main()


Extracted Tables:
--------------------------------------------------

Table 1:
                                                                                                                                                                                                                                                                                                                                           0                                                                                                                                                  1                                                                                                                        2
0  Name :MRS P J & SWATI L & J P GADAMobile No:98XXXXX156Email ID:XXXXXXXXada@gmail.comBilling Address :0-4-B,FLOOR-2,PLOT-3A,GURDEVI MANSION,K ASUBRAMANYAM MARG,BRAHMANWADA KINGSCIRCLE,MATUNGA,MUMBAI-400019Power Supply Address :0-4-B,FLOOR-2,PLOT-3A,GURDEVI MANSION,K ASUBRAMANYAM MARG,BRAHMANWADA KINGSCIRCLE,MATUNGA,

In [ ]:
!pip install pdfplumber

In [1]:
!apt-get install -y poppler-utils
!apt-get install -y tesseract-ocr
!pip install pytesseract
!pip install camelot-py[cv]
!pip install pdf2image

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 18 not upgraded.
Need to get 186 kB of archives.
After this operation, 696 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.6 [186 kB]
Fetched 186 kB in 0s (714 kB/s)
Selecting previously unselected package poppler-utils.
(Reading database ... 124926 files and directories currently installed.)
Preparing to unpack .../poppler-utils_22.02.0-2ubuntu0.6_amd64.deb ...
Unpacking poppler-utils (22.02.0-2ubuntu0.6) ...
Setting up poppler-utils (22.02.0-2ubuntu0.6) ...
Processing triggers for man-db (2.10.2-1) ...
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
T

In [ ]:
import camelot
import json
import re
from datetime import datetime
import pandas as pd
from pdf2image import convert_from_path
import pytesseract


In [34]:
# import camelot
# import json
# import re
# from datetime import datetime
# import pandas as pd
# from pdf2image import convert_from_path
# import pytesseract

class PDFExtractor:
    def __init__(self, file_path, pages):

        self.file_path = file_path
        self.pages = pages
        self.text = ""

    def extract_text_with_ocr(self):
        """Extract text from PDF pages using OCR and append to self.text."""
        all_text = ""
        for page in self.pages:
            images = convert_from_path(self.file_path, first_page=page, last_page=page)
            page_text = "\n".join([pytesseract.image_to_string(image) for image in images])
            all_text += f"\nPage {page} Text:\n{page_text}"
        self.text = all_text
        return self.text

    def extract_tables_from_pdf(self):
        """Extract and format tables from PDF"""
        tables = camelot.read_pdf(
            self.file_path,
            pages=",".join(map(str, self.pages)),
            flavor='lattice',
            line_scale=40,
            strip_text='\n'
        )

        formatted_tables = []
        for i, table in enumerate(tables):
            df = table.df  # Convert table to pandas DataFrame
            table_dict = {
                'table_number': i + 1,
                'rows': len(df),
                'cols': len(df.columns),
                'data': df.values.tolist()
            }
            formatted_tables.append(table_dict)

        return tables, formatted_tables

    def get_units_consumed(self, text):


        units_pattern_1 = r'(\d{1,4})\s*([A-Za-z]+-\d{2})'  # Format: Units first, then month
        units_pattern_2 = r'([A-Za-z]+-\d{2})\s*(\d{1,4})'  # Format: Month first, then units

        matches_1 = re.findall(units_pattern_1, text)
        matches_2 = re.findall(units_pattern_2, text)

        valid_months = [
            "Jan", "Feb", "Mar", "Apr", "May", "Jun",
            "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"
        ]

        unique_entries_1 = set()
        unique_entries_2 = set()

        for units, month in matches_1:

            month_part = month.split('-')[0]


            if month_part in valid_months:

                unique_entries_1.add((int(units), month))

        for month, units in matches_2:
            month_part = month.split('-')[0]
            if month_part in valid_months:
                unique_entries_2.add((month, int(units)))

        formatted_entries_1 = [{"units": units, "month": month} for units, month in unique_entries_1]
        formatted_entries_2 = [{"month": month, "units": units} for month, units in unique_entries_2]

        return {
            "units_first": formatted_entries_1,
            "month_first": formatted_entries_2
        }

    def extract_bill_period(self, text):
        """Extract bill period using regex pattern."""
        bill_period_pattern = r'Bill Period\s*:?\s*(\d{2}/\d{2}/\d{4})\s*-\s*(\d{2}/\d{2}/\d{4})'
        bill_period_match = re.search(bill_period_pattern, text)
        return {
            "start_date": bill_period_match.group(1) if bill_period_match else "",
            "end_date": bill_period_match.group(2) if bill_period_match else ""
        }

    def extract_fields_from_pdf(self):
        """Extract structured key-value data from tables, OCR text, bill period, and units consumed."""
        tables, formatted_tables = self.extract_tables_from_pdf()

        # Extract OCR text
        self.extract_text_with_ocr()

        all_text = self.text
        for table in tables:
            for index, row in table.df.iterrows():
                all_text += " ".join(row.astype(str)) + " "

        patterns = {
        'name': r'Name\s*:?\s*([^:\n]+?)(?=\s*(?:Mobile|Email|Billing|$))',
        'billing_address': r'Billing Address\s*:?\s*([^:\n]+?)(?=\s*(?:Power|Email|$))',
        "power_supply_address": r"Power Supply Address\s*:?\s*([^:\n]+?\w+-\d{6})",
        'mobile_no': r'Mobile No\s*:?\s*([\d\sX]+)',
        "email_id": r"Email ID\s*:\s*([a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{3})",
        "bill_for": r"Bill For\s*:\s*([A-Za-z]+\s*-\s*\d{4})",
        "date_of_bill": r"Date of Bill\s*:\s*([\d/]+)",
        "invoice_no": r"Invoice No\s*\.?\s*:\s*(\d+)",
        'book_folio_no': r'Book Folio No\.?\s*:?\s*([\d-]+)',
        'consumer_no': r'Consumer No\.?\s*:?\s*(\d{3}-\d{3}-\d{3}\*?\d)',
        'c_a_no': r'C\.?A\.?\s*No\.?\s*:?\s*([\d-]+)',
        'cycle': r'Cycle\s*:?\s*(\w+)(?=\s*(?:Type|$))',
        'type_of_supply': r'Type of Supply\s*:?\s*([^:\n]+?)(?=\s*(?:Service|$))',
        'service_no': r'Service No\.?\s*:?\s*([\w-]+)(?=\s*(?:Installation No.|$))',
        'installation_no': r'Installation No\.?\s*:?\s*([\w-]+)(?=\s*(?:Sanctioned|$))',
        'sanctioned_load': r'Sanctioned Load\s*:?\s*([\d.]+)\s*(?:KW|KVA)?',
        'security_deposit': r'Security Deposit\s*:?\s*(?:₹|Rs\.?)?\s*([\d.]+)',
        'last_payment_received': r'Last Payment Received\s*:?\s*(?:₹|Rs\.?)?\s*([\d.]+)',
        'last_payment_received_date': r'Last Payment Received Date\s*:?\s*(\d{2}/\d{2}/\d{4})',
        'tariff': r'Tariff\s*:?\s*(\w+(?:\s+\w+)*(?=\s*(?:Category|$)))',
        'category': r'Category\s*:?\s*(\w+)(?=\s*(?:Ward|$))',
        'ward': r'Ward\s*:?\s*([A-Z]+)',
        'current_bill_amount': r'Current Bill Amount`.*?\n\s*([\d.]+)',
        'past_dues': r'Past Dues ₹.*?\n\s*([\d.]+)',
        'due_date': r'Due Date.*?\n\s*(\d{2}/\d{2}/\d{4})',
        'bill_amount_before_due': r'Bill Amount Before Due Date.*?\n\s*([\d.]+)',
        'bill_amount_after_due': r'Bill Amount After Due Date.*?\n\s*([\d.]+)'
        }

        print("\nExtracted OCR & Table Text:\n", all_text)

        # Extract fields using regex
        extracted_data = {field: re.search(pattern, all_text, re.IGNORECASE).group(1).strip() if re.search(pattern, all_text, re.IGNORECASE) else "" for field, pattern in patterns.items()}


        extracted_data["bill_period"] = self.extract_bill_period(all_text)
        extracted_data["units_consumed"] = self.get_units_consumed(all_text)
        extracted_data["tables"] = formatted_tables
        extracted_data["ocr_text"] = self.text

        return extracted_data

def main():
    file_path = "/content/STL_BEST_046_NA_578344024_28-Jan-25.pdf"
    pages = [1, 2]


    extractor = PDFExtractor(file_path, pages)

    try:
        extracted_data = extractor.extract_fields_from_pdf()

        # Save extracted data to JSON
        output_file = 'extracted_fields.json'
        with open(output_file, 'w', encoding='utf-8') as f:
            json.dump(extracted_data, f, indent=4, ensure_ascii=False)

        print("\nExtracted Fields:")
        print(json.dumps(extracted_data, indent=2))

    except Exception as e:
        print(f"Error processing PDF: {str(e)}")


if __name__ == "__main__":
    main()



Extracted Fields:
{
  "name": "Bill For :  Jan-2025 Date of Bill: 28/01/2025 Invoice No. : 501742549014]\nSUYOG TELEMATICS LIMITED Book Folio N C N",
  "billing_address": "Type of Supply; 1p Bill Period : 20/12/2024 -\n17,FLOOR-G+TERRACE, EUROPEAN CEMETERY, SHAIKH Service No ny.\n\na MISREE ROAD,OPP WALCHAND BUNGLOW, ANTOP. . 878003-X-x . 21/01/2025\n\nB | HILL,MUMBAI-400037",
  "power_supply_address": "5 ity D it Ward\n\n6 | 17,FLOOR-G+TERRACE, EUROPEAN CEMETERY, SHAIKH ecurity Deposit :  11500.00 ar + FN\n\n4% | MISREE ROAD,OPP WALCHAND BUNGLOW, ANTOP = 5\n\n2 HILL, MUMBAI-400037",
  "mobile_no": "98XXXXX890",
  "email_id": "XXXXXXXXXXXXXXXXXXXXXXXXXics@gmail.com",
  "bill_for": "Jan-2025",
  "date_of_bill": "28/01/2025",
  "invoice_no": "501742549014",
  "book_folio_no": "",
  "consumer_no": "",
  "c_a_no": "",
  "cycle": "4",
  "type_of_supply": "",
  "service_no": "",
  "installation_no": "",
  "sanctioned_load": "",
  "security_deposit": "",
  "last_payment": "",
  "last_payment

In [38]:
# import camelot
# import json
# import re
# from datetime import datetime
# import pandas as pd
# from pdf2image import convert_from_path
# import pytesseract

class PDFExtractor:
    def __init__(self, file_path, pages):
        self.file_path = file_path
        self.pages = pages
        self.text = ""

    def extract_text_with_ocr(self):
        """Extract text from PDF pages using OCR and append to self.text."""
        all_text = ""
        for page in self.pages:
            images = convert_from_path(self.file_path, first_page=page, last_page=page)
            page_text = "\n".join([pytesseract.image_to_string(image) for image in images])
            all_text += f"\nPage {page} Text:\n{page_text}"
        self.text = all_text
        return self.text

    def extract_tables_from_pdf(self):
        """Extract and format tables from PDF"""
        tables = camelot.read_pdf(
            self.file_path,
            pages=",".join(map(str, self.pages)),
            flavor='lattice',
            line_scale=40,
            strip_text='\n'
        )

        formatted_tables = []
        for i, table in enumerate(tables):
            df = table.df  # Convert table to pandas DataFrame
            table_dict = {
                'table_number': i + 1,
                'rows': len(df),
                'cols': len(df.columns),
                'data': df.values.tolist()
            }
            formatted_tables.append(table_dict)

        return tables, formatted_tables

    def get_units_consumed(self, text):
        units_pattern_1 = r'(\d{1,4})\s*([A-Za-z]+-\d{2})'  # Format: Units first, then month
        units_pattern_2 = r'([A-Za-z]+-\d{2})\s*(\d{1,4})'  # Format: Month first, then units

        matches_1 = re.findall(units_pattern_1, text)
        matches_2 = re.findall(units_pattern_2, text)

        valid_months = [
            "Jan", "Feb", "Mar", "Apr", "May", "Jun",
            "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"
        ]

        unique_entries_1 = set()
        unique_entries_2 = set()

        for units, month in matches_1:
            month_part = month.split('-')[0]
            if month_part in valid_months:
                unique_entries_1.add((int(units), month))

        for month, units in matches_2:
            month_part = month.split('-')[0]
            if month_part in valid_months:
                unique_entries_2.add((month, int(units)))

        formatted_entries_1 = [{"units": units, "month": month} for units, month in unique_entries_1]
        formatted_entries_2 = [{"month": month, "units": units} for month, units in unique_entries_2]

        return {
            "units_first": formatted_entries_1,
            "month_first": formatted_entries_2
        }

    def extract_bill_period(self, text):
        """Extract bill period using regex pattern."""
        bill_period_pattern = r'Bill Period\s*:?\s*(\d{2}/\d{2}/\d{4})\s*-\s*(\d{2}/\d{2}/\d{4})'
        bill_period_match = re.search(bill_period_pattern, text)
        return {
            "start_date": bill_period_match.group(1) if bill_period_match else "",
            "end_date": bill_period_match.group(2) if bill_period_match else ""
        }

    def extract_fields_from_pdf(self):
        """Extract structured key-value data from tables, OCR text, bill period, and units consumed."""
        tables, formatted_tables = self.extract_tables_from_pdf()

        # Extract OCR text
        self.extract_text_with_ocr()

        all_text = self.text
        for table in tables:
            for index, row in table.df.iterrows():
                all_text += " ".join(row.astype(str)) + " "

        patterns = {
            'name': r'Name\s*:?\s*([^:\n]+?)(?=\s*(?:Mobile|Email|Billing|$))',
            'billing_address': r'Billing Address\s*:?\s*([^:\n]+?)(?=\s*(?:Power|Email|$))',
            "power_supply_address": r"Power Supply Address\s*:?\s*([^:\n]+?\w+-\d{6})",
            'mobile_no': r'Mobile No\s*:?\s*([\d\sX]+)',
            "email_id": r"Email ID\s*:\s*([a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{3})",
            "bill_for": r"Bill For\s*:\s*([A-Za-z]+\s*-\s*\d{4})",
            "date_of_bill": r"Date of Bill\s*:\s*([\d/]+)",
            "invoice_no": r"Invoice No\s*\.?\s*:\s*(\d+)",
            'book_folio_no': r'Book Folio No\.?\s*:?\s*([\d-]+)',
            'consumer_no': r'Consumer No\.?\s*:?\s*(\d{3}-\d{3}-\d{3}\*?\d)',
            'c_a_no': r'C\.?A\.?\s*No\.?\s*:?\s*([\d-]+)',
            'cycle': r'Cycle\s*:?\s*(\w+)(?=\s*(?:Type|$))',
            'type_of_supply': r'Type of Supply\s*:?\s*([^:\n]+?)(?=\s*(?:Service|$))',
            'service_no': r'Service No\.?\s*:?\s*([\w-]+)(?=\s*(?:Installation No.|$))',
            'installation_no': r'Installation No\.?\s*:?\s*([\w-]+)(?=\s*(?:Sanctioned|$))',
            'sanctioned_load': r'Sanctioned Load\s*:?\s*([\d.]+)\s*(?:KW|KVA)?',
            'security_deposit': r'Security Deposit\s*:?\s*(?:₹|Rs\.?)?\s*([\d.]+)',
            'last_payment_received': r'Last Payment Received\s*:?\s*(?:₹|Rs\.?)?\s*([\d.]+)',
            'last_payment_received_date': r'Last Payment Received Date\s*:?\s*(\d{2}/\d{2}/\d{4})',
            'tariff': r'Tariff\s*:?\s*(\w+(?:\s+\w+)*(?=\s*(?:Category|$)))',
            'category': r'Category\s*:?\s*(\w+)(?=\s*(?:Ward|$))',
            'ward': r'Ward\s*:?\s*([A-Z]+)',
            'current_bill_amount': r'Current Bill Amount`.*?\n\s*([\d.]+)',
            'past_dues': r'Past Dues ₹.*?\n\s*([\d.]+)',
            'due_date': r'Due Date.*?\n\s*(\d{2}/\d{2}/\d{4})',
            'bill_amount_before_due': r'Bill Amount Before Due Date.*?\n\s*([\d.]+)',
            'bill_amount_after_due': r'Bill Amount After Due Date.*?\n\s*([\d.]+)'
        }

       # print("\nExtracted OCR & Table Text:\n", all_text)

        extracted_data = {field: re.search(pattern, all_text, re.IGNORECASE).group(1).strip()
                         if re.search(pattern, all_text, re.IGNORECASE) else ""
                         for field, pattern in patterns.items()}

        # Extract table-specific fields (table_number=2)
        target_fields_mapping = {
            "Current Bill Amount`": "current_bill_amount",
            "Past Dues`": "past_dues",
            "Due Date*": "due_date",
            "Bill Amount Before Due Date`": "bill_amount_before_due",
            "Bill Amount After Due Date**`": "bill_amount_after_due"
        }

        for table in formatted_tables:
            if table['table_number'] == 2 and table['rows'] == 2 and table['cols'] == 5:
                headers = table['data'][0]
                values = table['data'][1]
                for header, value in zip(headers, values):
                    cleaned_header = header.strip("`*")

                    for original_header, field_name in target_fields_mapping.items():
                        if cleaned_header == original_header.strip("`*"):
                            extracted_data[field_name] = value
                            break


        extracted_data["bill_period"] = self.extract_bill_period(all_text)
        #extracted_data["units_consumed"] = self.get_units_consumed(all_text)
        extracted_data["tables"] = formatted_tables
        extracted_data["ocr_text"] = self.text

        return extracted_data


def main():
    file_path = "/content/604571021_2408.pdf"
    pages = [1, 2]


    extractor = PDFExtractor(file_path, pages)

    try:
        extracted_data = extractor.extract_fields_from_pdf()

        # Save extracted data to JSON
        output_file = 'extracted_fields.json'
        with open(output_file, 'w', encoding='utf-8') as f:
            json.dump(extracted_data, f, indent=4, ensure_ascii=False)

        print("\nExtracted Fields:")
        print(json.dumps(extracted_data, indent=2))

    except Exception as e:
        print(f"Error processing PDF: {str(e)}")


if __name__ == "__main__":
    main()



Extracted Fields:
{
  "name": "MRS P J & SWATI L & J P GADA",
  "billing_address": "0-4-B,FLOOR-2,PLOT-3A,GURDEVI MANSION,K ASUBRAMANYAM MARG,BRAHMANWADA KINGSCIRCLE,MATUNGA,MUMBAI-400019",
  "power_supply_address": "0-4-B,FLOOR-2,PLOT-3A,GURDEVI MANSION,K ASUBRAMANYAM MARG,BRAHMANWADA KINGSCIRCLE,MATUNGA,MUMBAI-400019",
  "mobile_no": "98XXXXX156",
  "email_id": "XXXXXXXXada@gmail.com",
  "bill_for": "Aug-2024",
  "date_of_bill": "16/08/2024",
  "invoice_no": "408604571021",
  "book_folio_no": "604571",
  "consumer_no": "604-571-021*1",
  "c_a_no": "1890975",
  "cycle": "09",
  "type_of_supply": "3P",
  "service_no": "312229-X-X",
  "installation_no": "0540489",
  "sanctioned_load": "18.980",
  "security_deposit": "3856.00",
  "last_payment_received": "10.00",
  "last_payment_received_date": "02/08/2024",
  "tariff": "LT I B",
  "category": "RESIDENTIAL",
  "ward": "FN",
  "current_bill_amount": "1165.05",
  "past_dues": "7.71",
  "due_date": "09/09/2024",
  "bill_amount_before_due":